In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

 #importing the files
movies = pd.read_csv("Ressources/movies.csv",encoding="Latin1")
Ratings = pd.read_csv("Ressources/ratings.csv")
Tags = pd.read_csv("Ressources/tags.csv",encoding="Latin1")

Mean = Ratings.groupby(by="userId",as_index=False)['rating'].mean()
Rating_avg = pd.merge(Ratings,Mean,on='userId')
Rating_avg['adg_rating']=Rating_avg['rating_x']-Rating_avg['rating_y']

check = pd.pivot_table(Rating_avg,values='rating_x',index='userId',columns='movieId')
final = pd.pivot_table(Rating_avg,values='adg_rating',index='userId',columns='movieId')

# Replacing NaN by Movie Average
final_movie = final.fillna(final.mean(axis=0))

# Replacing NaN by user Average
final_user = final.apply(lambda row: row.fillna(row.mean()), axis=1)

# user similarity on replacing NAN by user avg
b = cosine_similarity(final_user)
np.fill_diagonal(b, 0 )
similarity_with_user = pd.DataFrame(b,index=final_user.index)
similarity_with_user.columns=final_user.index

# user similarity on replacing NAN by item(movie) avg
cosine = cosine_similarity(final_movie)
np.fill_diagonal(cosine, 0 )
similarity_with_movie = pd.DataFrame(cosine,index=final_movie.index)
similarity_with_movie.columns=final_user.index

def find_n_neighbours(df,n):
    #order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
        .iloc[:n].index, 
        index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

# top 30 neighbours for each user
sim_user_30_m = find_n_neighbours(similarity_with_movie,30)

def get_user_similar_movies( user1, user2 ):
    common_movies = Rating_avg[Rating_avg.userId == user1].merge(
    Rating_avg[Rating_avg.userId == user2],
    on = "movieId",
    how = "inner" )
    return common_movies.merge( movies, on = 'movieId' )

a = get_user_similar_movies(370,86309)
a = a.loc[ : , ['rating_x_x','rating_x_y','title']]


Rating_avg = Rating_avg.astype({"movieId": str})
Movie_user = Rating_avg.groupby(by = 'userId')['movieId'].apply(lambda x:','.join(x))

def User_item_score1(user):
    Movie_seen_by_user = check.columns[check[check.index==user].notna().any()].tolist()
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    d = Movie_user[Movie_user.index.isin(b)]
    l = ','.join(d.values)
    Movie_seen_by_similar_users = l.split(',')
    Movies_under_consideration = list(set(Movie_seen_by_similar_users)-set(list(map(str, Movie_seen_by_user))))
    Movies_under_consideration = list(map(int, Movies_under_consideration))
    score = []
    for item in Movies_under_consideration:
        c = final_movie.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        avg_user = Mean.loc[Mean['userId'] == user,'rating'].values[0]
        index = f.index.values.squeeze().tolist()
        corr = similarity_with_movie.loc[user,index]
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['adg_score','correlation']
        fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = avg_user + (nume/deno)
        score.append(final_score)
    data = pd.DataFrame({'movieId':Movies_under_consideration,'score':score})
    top_20_recommendation = data.sort_values(by='score',ascending=False).head(20)
    Movie_Name = top_20_recommendation.merge(movies, how='inner', on='movieId')
    Movie_Names = Movie_Name.title.values.tolist()
    return Movie_Names


# Similarity Measure (COS)
def GetRecommendations(id):
    return User_item_score1(id)

In [4]:
similarity_with_movie

userId,1,2,3,4,5,6,7,8,9,10,...,603,604,605,606,607,608,609,610,611,612
userId,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.988282,0.978404,0.964217,0.986818,0.970454,0.971640,0.987467,0.986381,0.973411,...,0.917916,0.983977,0.978636,0.959689,0.976117,0.932800,0.989379,0.952769,0.987754,0.989423
2,0.988282,0.000000,0.987140,0.971164,0.995793,0.979891,0.981851,0.995167,0.995107,0.981300,...,0.929080,0.993014,0.988205,0.968865,0.983617,0.940219,0.997957,0.963109,0.996443,0.998126
3,0.978404,0.987140,0.000000,0.961233,0.985178,0.970770,0.971930,0.985139,0.985262,0.971478,...,0.921427,0.983192,0.978361,0.957064,0.974112,0.930647,0.988085,0.954259,0.986586,0.988251
4,0.964217,0.971164,0.961233,0.000000,0.968636,0.955183,0.958873,0.970087,0.969156,0.959639,...,0.903111,0.967103,0.963959,0.942696,0.958887,0.911584,0.972097,0.935859,0.970686,0.972325
5,0.986818,0.995793,0.985178,0.968636,0.000000,0.978366,0.980009,0.992904,0.993493,0.979176,...,0.928120,0.991065,0.986089,0.967227,0.982364,0.938348,0.996583,0.960849,0.994828,0.996508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608,0.932800,0.940219,0.930647,0.911584,0.938348,0.920307,0.924798,0.939781,0.939037,0.921722,...,0.873531,0.935649,0.932003,0.912633,0.928790,0.000000,0.941616,0.906351,0.939992,0.941578
609,0.989379,0.997957,0.988085,0.972097,0.996583,0.980977,0.982444,0.996521,0.996061,0.982020,...,0.930112,0.994488,0.988954,0.969851,0.984565,0.941616,0.000000,0.963318,0.997397,0.999082
610,0.952769,0.963109,0.954259,0.935859,0.960849,0.946972,0.948312,0.961407,0.961813,0.945597,...,0.896481,0.958719,0.954812,0.934619,0.949232,0.906351,0.963318,0.000000,0.961697,0.963849
